In [1]:
import sys
#conda install --prefix {sys.prefix} tweepy
#import tweepy
#from tweepy.streaming import StreamListener
#from tweepy import OAuthHandler
#from tweepy import Stream
import tweepy
import jsonlines
import json
import pandas as pd
import datetime

class TweetsAnalyzer:   
    consumerAPIKey = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX" 
    consumerSecretKey = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
    accessToken = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
    accessSecret = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
    
    def __init__(self):
        self.parsedResultDictionary = {'Text' : [], 'Date': [],'Time': [], 'Likes': [], 'Retweets': [], 'Images': []}
    
    # Helper method to write in file
    def writeTweetsInJsonlFile(self, tweets, filePath):
        with jsonlines.open(filePath, mode='w') as writer:
            for li in tweets:
                try:
                    writer.write(li._json)
                except:
                    continue
                    
    # Helper method for parsing tweets
    def jsonlParser(self, tweet):
        if 'retweet_count' in tweet:
            li = self.parsedResultDictionary['Retweets'] 
            li.append(tweet['retweet_count'])
            self.parsedResultDictionary['Retweets'] = li
        else:
            li = self.parsedResultDictionary['Retweets']
            li.append('-')
            self.parsedResultDictionary['Retweets'] = li
        if 'favorite_count' in tweet:
            li = self.parsedResultDictionary['Likes'] 
            li.append(tweet['favorite_count'])
            self.parsedResultDictionary['Likes'] = li
        else:
            li = self.parsedResultDictionary['Likes']
            li.append('-')
            self.parsedResultDictionary['Likes'] = li
        if 'created_at' in tweet:        
            datetimeParsed = datetime.datetime.strptime(tweet['created_at'], '%a %b %d %H:%M:%S %z %Y')
            li = self.parsedResultDictionary['Date'] 
            li.append(datetimeParsed.date())
            self.parsedResultDictionary['Date'] = li        
            li = self.parsedResultDictionary['Time'] 
            li.append(datetimeParsed.time())
            self.parsedResultDictionary['Time'] = li
        else:
            li = self.parsedResultDictionary['Date']
            li.append('-')
            self.parsedResultDictionary['Date'] = li
            li = self.parsedResultDictionary['Time'] 
            li.append(datetimeParsed.time())
            self.parsedResultDictionary['Time'] = li
        if 'text' in tweet:
            li = self.parsedResultDictionary['Text'] 
            li.append(tweet['text'])
            self.parsedResultDictionary['Text'] = li
        else:
            li = self.parsedResultDictionary['Text']
            li.append('None')
            self.parsedResultDictionary['Text'] = li
        if 'entites' in tweet:
            li = self.parsedResultDictionary['Images'] 
            media = status.entities.get('media', [])
            if(len(media) > 0):
                li.append(len(media))
            else:
                li.append('None')            
            self.parsedResultDictionary['Images'] = li
        else:
            li = self.parsedResultDictionary['Images']
            li.append('None')
            self.parsedResultDictionary['Images'] = li
            
        return self.parsedResultDictionary
    
    # Printing tweets data in tabular form
    def printTweetDataFromJsonlFile(self, filePath):
        with jsonlines.open(filePath) as reader:
            for obj in reader:
                tweet = json.loads(json.dumps(obj))
                parsedResult = self.jsonlParser(tweet)
            
        df = pd.DataFrame(parsedResult)
        print (df)
    
    # Fetching 'threshold' (maximum number) tweets by given twitter handle, and writing to given file
    def fetchTweetsByTwitterHandleInFile(self, username, threshold, filePath):
        # Authorization         
        auth = tweepy.OAuthHandler(self.consumerAPIKey, self.consumerSecretKey)
        auth.set_access_token(self.accessToken, self.accessSecret)
        api = tweepy.API(auth)
        tweets = api.user_timeline(id=username, count = 10)
        self.writeTweetsInJsonlFile(tweets, filePath)   

# Driver code 
if __name__ == '__main__': 
    
    analyzer = TweetsAnalyzer()
    filePath = 'output.jsonl'
    threshold = 10
    
    print ('Loading data...')
    ## twitter handle for the user whose tweets are to be extracted in the given file.
    analyzer.fetchTweetsByTwitterHandleInFile("midasIIITD", threshold, filePath)
    
    ## printing tweets in tabular form
    analyzer.printTweetDataFromJsonlFile( filePath )
    
    

Loading data...
         Date Images  Likes  Retweets  \
0  2019-04-09   None      0         9   
1  2019-04-09   None      0        35   
2  2019-04-09   None      0        12   
3  2019-04-08   None      0        15   
4  2019-04-08   None     14         2   
5  2019-04-08   None      5         0   
6  2019-04-07   None      0         0   
7  2019-04-07   None      0         0   
8  2019-04-07   None      1         1   
9  2019-04-07   None      5         2   

                                                Text      Time  
0  RT @IIITDelhi: We are delighted to share that ...  16:45:07  
1  RT @Harvard: Professor Jelani Nelson founded A...  05:04:27  
2  RT @emnlp2019: For anyone interested in submit...  05:04:11  
3  RT @multimediaeval: Announcing the 2019 MediaE...  19:38:09  
4  Many Congratulations to @midasIIITD student, S...  07:08:12  
5  @midasIIITD thanks all students who have appea...  03:27:42  
6  @himanchalchandr Meanwhile, complete CV/NLP ta...  14:17:29  
7  @sayangdi